<p align="center">
<b><font size="9" color="#000080">PORTICO PLANO ETABS</font></b>
<div align="center">
    <div style="display: inline-block;">
    <span style="font-size: 18px;position: relative; bottom: 13px;">Richard Cueto Calderon | Update :7/05/204 | API ETBAS | Python</span>
    </div>
</div>
</p>

In [75]:
import os
import sys
import comtypes.client
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Etabas

In [76]:
AttachToInstance = False
SpecifyPath = True
ProgramPath = "C:\Program Files\Computers and Structures\ETABS 19\ETABS.exe"
APIPath = r'D:\Users\User(D)\Documentos\GitHub\freeproyect\Etabs\Modelos'
if not os.path.exists(APIPath):
    try:
        os.makedirs(APIPath)
    except OSError:
        pass
ModelPath = APIPath + os.sep + 'Portico.edb'
helper = comtypes.client.CreateObject('ETABSv1.Helper')
helper = helper.QueryInterface(comtypes.gen.ETABSv1.cHelper)
if AttachToInstance:
    try:
        myETABSObject = helper.GetObject("CSI.ETABS.API.ETABSObject") 
    except (OSError, comtypes.COMError):
        print("No running instance of the program found or failed to attach.")
        sys.exit(-1)
else:
    if SpecifyPath:
        try:
            myETABSObject = helper.CreateObject(ProgramPath)
        except (OSError, comtypes.COMError):
            print("Cannot start a new instance of the program from " + ProgramPath)
            sys.exit(-1)
    else:
        try: 
            myETABSObject = helper.CreateObjectProgID("CSI.ETABS.API.ETABSObject") 
        except (OSError, comtypes.COMError):
            print("Cannot start a new instance of the program.")
            sys.exit(-1)
    myETABSObject.ApplicationStart()
SapModel = myETABSObject.SapModel
SapModel.InitializeNewModel()

0

## Modelo

In [77]:
# Creamos un nuevo modelo en blanco
ret=SapModel.File.NewBlank()

In [78]:
# Cambiamos unidades a ton_m
Tonf_m_C=12
ret = SapModel.SetPresentUnits(Tonf_m_C)

In [79]:
# Definicion de material concreto
fc=280
material_type=2
material_concreto="fc"+str(fc)+"kg/cm2" #fc=280kg/cm2
ret=SapModel.PropMaterial.SetMaterial(material_concreto, material_type)
# Definir propiedades mecanicas del material
E=15000*fc**0.5             #kg/cm2
v=0.2
CT=0.0000055
ret=SapModel.PropMaterial.SetMPIsotropic(material_concreto,E,v,CT)

In [80]:
# Definicion de material albañileria
fc=100
material_type=8
material_albañileria="fc"+str(fc)+"kg/cm2" #fc=280kg/cm2
ret=SapModel.PropMaterial.SetMaterial(material_albañileria, material_type)
# Definir propiedades mecanicas del material
E=150*fc**0.5             #kg/cm2
v=0.2
CT=0.0000055
ret=SapModel.PropMaterial.SetMPIsotropic(material_albañileria,E,v,CT)

In [81]:
# Creamos seccion de viga y columna
# Viga
b_v=0.3
h_v=0.6
nombre_viga='v '+str(b_v)+'x'+str(h_v)          #V 0.30x0.60
ret = SapModel.PropFrame.SetRectangle(nombre_viga, material_concreto, h_v, b_v)

# Columna
b_c=0.45
h_c=0.45
nombre_columna='v '+str(b_c)+'x'+str(h_c)          #V 0.45x0.45
ret = SapModel.PropFrame.SetRectangle(nombre_columna, material_concreto, h_c, b_c)

In [82]:
# definimos modificaciones
ModValue = [1, 1, 0, 1, 1, 1, 1, 1]
ret = SapModel.PropFrame.SetModifiers(nombre_viga, ModValue)
ret = SapModel.PropFrame.SetModifiers(nombre_columna, ModValue)

In [83]:
ret = SapModel.PropArea.SetSlab('Losa maciza',0,1,material_concreto,.15)

In [84]:
ret = SapModel.PropArea.SetWall('MURO ALBAÑILERIA', 1,1, material_albañileria,.142) 

**CREACION DE PORTICO**

Esta creacion ser de manera manual y mecanica

+ Creacion de Frame por coordendas
+ Ingresamos paramteros


In [85]:
h=3
L=4
L2=2
paños=10
pañosy=3
niveles=5

for k in range(niveles):
    for j in range(pañosy+1):
        #columnas
        for i in range(paños+1):
            FrameName = ' '
            [FrameName, ret] = SapModel.FrameObj.AddByCoord(i*L, j*L2, k*h,i*L, j*L2, (k+1)*h, FrameName, nombre_columna, '', 'Global')

        # vigas
        for i in range(paños):
            FrameName = ' '
            [FrameName, ret] = SapModel.FrameObj.AddByCoord(i*L, j*L2, (k+1)*h, (i+1)*L, j*L2, (k+1)*h, FrameName, nombre_viga, '', 'Global')

    for j in range(pañosy):   
        # transversales
        for i in range(paños+1):
            FrameName = ' '
            [FrameName, ret] = SapModel.FrameObj.AddByCoord(i*L, j*L2, (k+1)*h, i*L, (j+1)*L2, (k+1)*h, FrameName, nombre_viga, '', 'Global')


In [86]:
# Agregamos muro
for k in range(niveles):
    #xz
    nameWall=""
    x=[0,L,L,0]
    y=[0,0,0,0]
    z=[0,0,(k+1)*h,(k+1)*h]
    ret = SapModel.AreaObj.AddByCoord(4, x, y, z,nameWall,'MURO ALBAÑILERIA')
    #xz
    nameWall=""
    x=[(paños-1)*L,paños*L,paños*L,(paños-1)*L]
    y=[0,0,0,0]
    z=[0,0,(k+1)*h,(k+1)*h]
    ret = SapModel.AreaObj.AddByCoord(4, x, y, z,nameWall,'MURO ALBAÑILERIA')
    
    #yz
    nameWall=""
    x=[0,0,0,0]
    y=[0,L2,L2,0]
    z=[0,0,(k+1)*h,(k+1)*h]
    ret = SapModel.AreaObj.AddByCoord(4, x, y, z,nameWall,'MURO ALBAÑILERIA')
    #yz
    nameWall=""
    x=[paños*L,paños*L,paños*L,paños*L]
    y=[(pañosy-1)*L2,pañosy*L2,pañosy*L2,(pañosy-1)*L2]
    z=[0,0,(k+1)*h,(k+1)*h]
    ret = SapModel.AreaObj.AddByCoord(4, x, y, z,nameWall,'MURO ALBAÑILERIA')

In [89]:
#assign point object restraint at base
Restraint = [True, True, True, True, True, True]
for j in range(pañosy+1):
    for i in range(paños+1):
        Pointi,Pointj= ' ',' '
        [Pointi, Pointj, ret] = SapModel.FrameObj.GetPoints(str(i+1), Pointi, Pointj)
        ret = SapModel.PointObj.SetRestraint(Pointi, Restraint)

In [ ]:
# Creamos un LoadPattern
ret = SapModel.LoadPatterns.Add('FH', 8, 0, True)
for j in range(niveles):
    for i in range(paños):
        # Asignar carga distribuida
        ret = SapModel.FrameObj.SetLoadDistributed(str(paños+2+i+(2*paños+1)*j),'Dead',1,2,0,1,-0.4,-0.4,'Local')
        ret = SapModel.FrameObj.SetLoadDistributed(str(paños+2+i+(2*paños+1)*j),'Live',1,2,0,1,-0.25,-0.25,'Local')

In [ ]:

# Asignar carga puntual
PointLoadValue = [1,0,0,0,0,0]
Pointi=""
Pointj=""
[Pointi, Pointj, ret] = SapModel.FrameObj.GetPoints("1", Pointi, Pointj)
ret = SapModel.PointObj.SetLoadForce(Pointj,'FH',PointLoadValue)

In [ ]:
#refresh view, update (initialize) zoom
ret = SapModel.View.RefreshView(0, False)

In [ ]:
#save model
ret = SapModel.File.Save(ModelPath)

In [ ]:
#run model (this will create the analysis model)
ret = SapModel.Analyze.RunAnalysis()

# EXTRAER DATOS DEL ETABS

El codigo proporcionado nos permitira extraer cualquier tabla que se encuentre en el programa etabs.

El trabajo de esta sera identificar las tablas con las que queremos trabajar.

In [4]:
def conect_to_etabs():
    helper = comtypes.client.CreateObject('ETABSv1.Helper')
    helper = helper.QueryInterface(comtypes.gen.ETABSv1.cHelper)
    try:
        myETABSObject = helper.GetObject("CSI.ETABS.API.ETABSObject")
        print('Conected to ETABS model')
    except (OSError, comtypes.COMError):
        print("No running instance of the program found or failed to attach.")
        sys.exit(-1)
    SapModel = myETABSObject.SapModel
    return SapModel,myETABSObject,helper

SapModel,myETABSObject,helper=conect_to_etabs()

Conected to ETABS model


In [ ]:
name_table="Base Reactions"
name_table="Element Forces - Beams"
table=SapModel.DatabaseTables.GetTableForDisplayArray (name_table,GroupName="")
colums=table[2]
noOfRows=table[3]
val=np.array_split(table[4],noOfRows)
df=pd.DataFrame(val)
df.columns=colums
df

In [ ]:
df

In [ ]:
# df=df.drop(columns='FX')
# df=df.drop(columns='FY')
# df=df.drop(columns='MZ')
# df=df.drop(columns='X')
# df=df.drop(columns='Y')
# df=df.drop(columns='Z')

In [ ]:
df

## creamo diaphragm

In [90]:
ret=SapModel.Diaphragm.SetDiaphragm('D1',SemiRigid=True)

In [91]:
ret=SapModel.Diaphragm.SetDiaphragm('D2',False)
ret=SapModel.Diaphragm.SetDiaphragm('D3',False)
ret=SapModel.Diaphragm.SetDiaphragm('D4',False)
ret=SapModel.Diaphragm.SetDiaphragm('D5',False)
ret=SapModel.Diaphragm.SetDiaphragm('D6',False)


In [92]:
numberName=0
myName=[]
myLabel=[]
myStory=[]
lista=SapModel.AreaObj.GetLabelNameList(numberName,myName,myLabel,myStory)
print(lista[0])#cantidad elementos area
print(lista[1])#uniqueName
print(lista[2])#Label
print(lista[3])#story

20
('1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20')
('W1', 'W2', 'W3', 'W4', 'W5', 'W6', 'W7', 'W8', 'W9', 'W10', 'W11', 'W12', 'W13', 'W14', 'W15', 'W16', 'W17', 'W18', 'W19', 'W20')
('Story1', 'Story1', 'Story1', 'Story1', 'Story1', 'Story1', 'Story1', 'Story1', 'Story1', 'Story1', 'Story1', 'Story1', 'Story1', 'Story1', 'Story1', 'Story1', 'Story1', 'Story1', 'Story1', 'Story1')


In [94]:
colums=["uniqueName","label","story"]
df=pd.DataFrame(np.transpose([lista[1],lista[2],lista[3]]),columns=colums)
df

,uniqueName,label,story
0,1,W1,Story1
1,2,W2,Story1
2,3,W3,Story1
3,4,W4,Story1
4,5,W5,Story1
5,6,W6,Story1
6,7,W7,Story1
7,8,W8,Story1
8,9,W9,Story1
9,10,W10,Story1


In [95]:
#filtro de columna story
df=df[df["story"]=="Story1"]
df

,uniqueName,label,story
0,1,W1,Story1
1,2,W2,Story1
2,3,W3,Story1
3,4,W4,Story1
4,5,W5,Story1
5,6,W6,Story1
6,7,W7,Story1
7,8,W8,Story1
8,9,W9,Story1
9,10,W10,Story1


In [97]:
df=df[df["label"].str.startswith("W")]
df

,uniqueName,label,story
0,1,W1,Story1
1,2,W2,Story1
2,3,W3,Story1
3,4,W4,Story1
4,5,W5,Story1
5,6,W6,Story1
6,7,W7,Story1
7,8,W8,Story1
8,9,W9,Story1
9,10,W10,Story1


In [99]:
for uniqueName in df['uniqueName']:
    ret=SapModel.AreaObj.SetDiaphragm(uniqueName,'D1')

lista frames

In [100]:
lista_frames=SapModel.FrameObj.GetLabelNameList(numberName,myName,myLabel,myStory)
print(lista_frames[0])#cantidad elementos area
print(lista_frames[1])#uniqueName
print(lista_frames[2])#Label
print(lista_frames[3])#story

585
('1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', '100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '110', '111', '112', '113', '114', '115', '116', '117', '118', '119', '120', '121', '122', '123', '124', '125', '126', '127', '128', '129', '130', '131', '132', '133', '134', '135', '136', '137', '138', '139', '140', '141', '142', '143', '144', '145', '146', '147', '148', '149', '150', '151', '152', '153', '154', '155', '156', '157', '158

In [102]:
colums=["uniqueName","label","story"]
df=pd.DataFrame(np.transpose([lista_frames[1],lista_frames[2],lista_frames[3]]),columns=colums)
df

,uniqueName,label,story
0,1,C1,Story1
1,2,C2,Story1
2,3,C3,Story1
3,4,C4,Story1
4,5,C5,Story1
...,...,...,...
580,581,B361,Story1
581,582,B362,Story1
582,583,B363,Story1
583,584,B364,Story1


In [103]:

ret=SapModel.FrameObj.SetEndLengthOffset("312", False, 0.3, 0.3, 0.5)